<a href="https://colab.research.google.com/github/jorgel-mendes/HackDataSantanderIPYNB/blob/master/Hackathon_investiga%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caminhando ao Prospper

Esse notebook mostra exploração de dados utilizados que embasou a criação do Prospper, uma solução integrada com o CoPiloto Santander que busca empoderar donos de negócio repassando informação em um formato de simples utilização. 

A solução foi criada para o Hackathon Data Challenge do Santander do grupo **grupo 41?**.

Na nossa análise de compreensão do problema utilizamos dados do IBGE recolhidos no site do [IPEA](http://www.ipeadata.gov.br/Default.aspx) e uma [API em Python](https://github.com/GeneralMills/pytrends) para o google trends.

In [1]:
!pip install -q pytrends

In [2]:
import pandas as pd
import altair as alt
alt.renderers.enable(embed_options={'theme': 'googlecharts'})

from datetime import datetime
alt.renderers.enable('html')

RendererRegistry.enable('html')

# Análise do setor de Varejo em 2020



Pensando em como a crise afetou os pequenos negócios em geral, decidimos focar no setor de varejo, onde muitos estabelecimentos estão em risco de fechar e estão em buscas de soluções.

Foi utilizada a base de Indíce de varejo do IBGE para acompanhar o desempenho do setor nos ultimos anos. Os dados da base foram colhidos no IPEADATA, site do Instituto de Pesquisa Econômica Aplicada (IPEA).

Os dados representam os dados de venda do setor do varejo comparados com o ano de 2014. Basicamento o valor de 2014 é 100 e os outros números são baseados nele. Por exemplo, um valor de 101 no index significa vendas 1% maiores que o valor de vendas de varejo em 2014.

Mas comoo que nos importa é a tendência basta saber que pontos mais baixos significam pior desempenho e pontos mais altos desempenhos melhores.

In [3]:
pd.read_csv('https://raw.githubusercontent.com/jorgel-mendes/HackDataSantanderIPYNB/master/ipeadata%5B02-08-2020-01-36%5D.csv')

,Data,Vendas reais - varejo - índice dessaz. (média 2014 = 100) - - - Instituto Brasileiro de Geografia e Estatística- Pesquisa Mensal de Comércio (IBGE/PMC) - PMC12_IVVRSA12,Unnamed: 2
0,2000.01,49.5,NaN
1,2000.02,50.0,NaN
2,2000.03,50.3,NaN
3,2000.04,50.3,NaN
4,2000.05,50.7,NaN
...,...,...,...
240,2020.01,94.9,NaN
241,2020.02,95.6,NaN
242,2020.03,92.9,NaN
243,2020.04,77.8,NaN


In [4]:
ipea_varejo = pd.read_csv('https://raw.githubusercontent.com/jorgel-mendes/HackDataSantanderIPYNB/master/ipeadata%5B02-08-2020-01-36%5D.csv')
ipea_varejo.drop(ipea_varejo.columns[2], axis=1, inplace=True)
ipea_varejo.columns = ['Data', 'Vendas reais (varejo)']
ipea_varejo['Data'] = ipea_varejo['Data'].apply(
    lambda x: datetime.strptime(str(x), '%Y.%m'))

alt.Chart(
    ipea_varejo.query('Data > "2008-12-31"')
    ).mark_line().encode(
    x=alt.X('Data', title=''),
    y=alt.Y('Vendas reais (varejo)',
            title='Index de vendas de varejo',
            scale=alt.Scale(zero=False))
).properties(
    title="Index de vendas de varejo atingiu o ponto mais baixo em 10 anos"
)

alt.Chart(...)

Os dados do Ipea confirmam a necessidade de soluções na área de varejo já que houve o pior valor nos últimos anos. Pelo nosso parceiro, decidimos olhar soluções bancárias para o problema. Primeiro ver como está o desempenho delas atualmente e dependendo das descobertas procurar soluções.


# Análise do setor pelo Google Trends

Utilizando o pytrends podemos acessar em código o google trends para obter dados de pesquisa de usuários, ganhando uma noção de interesse através dos termos mais pesquisados. 

Primeiro olhando para alguns termos do setor bancário em geral e o parceiro do hackathon, o Santander.

In [5]:
from pytrends.request import TrendReq

pytrend = TrendReq(hl='BR', tz=180)

In [6]:
palavras_chave = ['financiamento',
            'financiamento santander',
            'santander', 'banco']

pytrend.build_payload(
     kw_list=palavras_chave,
     cat=0,
     timeframe='2019-01-01 2020-06-01',
     geo='BR',
     gprop='')

pytrend_data = pytrend.interest_over_time().drop(['isPartial'], axis=1)

alt.Chart(pytrend_data.reset_index(),
          title = ''
          ).transform_fold(
    palavras_chave,
    ).mark_line().encode(
    x=alt.X('date:T',
            title='',
            axis = alt.Axis(format=('%m/%y'))),
    y=alt.Y('value:Q', 
            title='Index de buscas no Google'),
    color=alt.Color('key:N', title='Termo')
).properties(
    title='Serviços bancários estão em alta em 2020'
)

alt.Chart(...)

Os termos ligados a bancos tiveram um alta esse ano como consequência da pandemia. O que mostra solução desse tipo já é procurada por negócios normalmente e portanto já é um canal que aproxima a solução dos afetados. Podendo até mesmo usar um produto já criado para implementar outro.

Agora vamos olhar mais para uns termos em específico. Começando com financiamento, onde veremos os termos sugeridos pelo google para essa pesquisa.





In [7]:
related_pytrend = pytrend.related_queries()
related_pytrend['financiamento']['rising'].head()['query']

0    solicitar pausa financiamento caixa
1          prorrogar financiamento caixa
2              pausa financiamento caixa
3                   pausar financiamento
4           prorrogação de financiamento
Name: query, dtype: object

Termos de financiamento estão populares no momento devido a renegociação de dívidas pela pandemia. 

Agora olhando mais sobre crédito.

In [8]:
palavras_chave = ['linha de crédito',
            'crédito para empresas']

pytrend.build_payload(
     kw_list=palavras_chave,
     cat=0,
     timeframe='2019-01-01 2020-06-01',
     geo='BR',
     gprop='')

pytrend_data = pytrend.interest_over_time().drop(['isPartial'], axis=1)

alt.Chart(pytrend_data.reset_index(),
          title = ''
          ).transform_fold(
    palavras_chave,
    ).mark_line().encode(
    x=alt.X('date:T',
            title='',
            axis = alt.Axis(format=('%m/%y'))),
    y=alt.Y('value:Q', 
            title='Index de buscas no Google'),
    color=alt.Color('key:N', title='Termo')
).properties(
    title='Pesquisas relacionadas a crédito cresceram durante a pandemia'
)

alt.Chart(...)

De novo confirmando que a demanda para produtos bancários está em alta, já que para muitos negócios eles significam a sobrevivência.

Agora que já confirmamos o setor bancário como um setor em alta no momento e ligado ao público mais afetado pela crise, podemos ver alguns produtos ainda mais de nicho.

O Santander possui alguns produtos direcionados a pequenos negócios, vamo ver como eles se comportaram e com a curva de 'crédito para empresas' para comparação do setor.

In [9]:
palavras_chave = ['santander prospera',
            'santander copiloto',
            'crédito para empresas']

pytrend.build_payload(
     kw_list=palavras_chave,
     cat=0,
     timeframe='2019-01-01 2020-06-01',
     geo='BR',
     gprop='')

pytrend_data = pytrend.interest_over_time().drop(['isPartial'], axis=1)

alt.Chart(pytrend_data.reset_index()
          ).transform_fold(
    palavras_chave,
    ).mark_line().encode(
    x=alt.X('date:T',
            axis = alt.Axis(format=('%m/%y'),
            title = '')),
    y=alt.Y('value:Q', 
            title='Index de buscas no Google'),
    color=alt.Color('key:N', title='Termo')
).properties(
    title='Copiloto e Prospera estão se popularizando mas há espaço de crescimento' 
)

alt.Chart(...)

O setor de crédito e financiamento para empresas está em alta mas as soluções Prospera e Copiloto ainda podem ganhar espaço e atingir mais pequenas empresas.

Decidimos então aproveitar essa oportunidade para criar uma solução em conjunto com esses negócios. Aproveitando a alta do setor bancário e também ajudando a impulsioná-los.

# Conclusão

Com essa análise nossos esforços foram direcionados para um serviço que ajude pequenas empresas a entenderem melhor seu público e, assim, ajudando-as a sobreviver. A solução foi feita em associação a um produto já existente do Banco Santander.

[Solução](www.link para solução)